In [1]:
# disable warnings up front for a cleaner experience
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
 

# League of Legends

*League of Legends* or *LoL* is a MOBA developed by Riot Games, captivating players with its strategic gameplay and vibrant esports scene. Originating as a standalone title, it has grown to become one of the most prominent games globally. The pinnacle of competitive play in League of Legends is the *World Championship* (Worlds) an annual tournament that draws attention from millions of fans. The prize pool fluctuates, with the champions taking home substantial amounts (e.g., $2.34 million in 2022, $2.34 million in 2021, $2.34 million in 2019, etc.). Teams vie for a spot in Worlds through regional leagues and qualifiers.

League of Legends is renowned for its depth and demanding learning curve. The game has a roster of over 150 champions, each equipped with a unique set of abilities. Teams are composed of five champions each, and with a multitude of items and strategic choices, the potential combinations are staggering. The complexity amplifies when considering lane assignments, itemization strategies, and other tactical nuances. Mastering League of Legends requires an extensive understanding of the game's intricacies, in addition to the mechanical skills required to control a champion, collaborate with teammates, and adapt to opponents' strategies.

Attempting to quantify the game's complexity and meta is a formidable task, yet we can gain insights by analyzing extensive match data from the World Championship throughout the years.
The website [Game of League Esports](https://gol.gg/esports/home/) has an immense amount of data regarding almost all professional LoL Tournaments. While most of the data is free, the website does have premium options for more data such as betting, and analytical information. 

I am going to use the match data from the World Championship Series and focusing on the World Championship 2023 Series that just concluded in November:

* [World Championship 2015](https://gol.gg/champion/list/season-S5/split-ALL/tournament-World%20Championship%202015/) (2015-10-01 - 2015-10-31)
* [World Championship 2016](https://gol.gg/champion/list/season-S6/split-ALL/tournament-World%20Championship%202016/) (2016-09-29 - 2016-10-29)
* [World Championship 2017](https://gol.gg/champion/list/season-S7/split-ALL/tournament-World%20Championship%202017/) (2017-09-23 - 2017-11-04)
* [World Championship 2018](https://gol.gg/champion/list/season-S8/split-ALL/tournament-World%20Championship%202018/) (2018-10-01 - 2018-11-03)
* [World Championship 2019](https://gol.gg/champion/list/season-S9/split-ALL/tournament-World%20Championship%202019/) (2019-10-02 - 2019-11-10)
* [World Championship 2020](https://gol.gg/champion/list/season-S10/split-ALL/tournament-World%20Championship%202020/) (2020-09-25 - 2020-10-31)
* [World Championship 2021](https://gol.gg/champion/list/season-S11/split-ALL/tournament-World%20Championship%202021/) (2021-10-05 - 2021-11-06)
* [World Championship 2022](https://gol.gg/champion/list/season-S12/split-ALL/tournament-World%20Championship%202022/) (2022-09-29 - 2022-11-05)
* [Worlds Main Event 2023](https://gol.gg/champion/list/season-ALL/split-ALL/tournament-Worlds%20Main%20Event%202023/) (2023-10-10 - 2023-11-19)

We also have a file that shows data regarding the Top Laners in the Worlds Main Event 2023 that will be used to observe the differences in the professional players for this tournament.
* [Worlds Main Event 2023 Top Lane](https://gol.gg/players/list/season-S13/split-ALL/tournament-Worlds%20Main%20Event%202023/)

There is one more file that shows all matches played in the Worlds Main Event 2023, specifically the teams that played, the winning team, and the champions picked by each team.
* [2023 Season World Champion/Main Event](https://lol.fandom.com/Special:RunQuery/MatchHistoryGame?MHG%5Bpreload%5D=Tournament&MHG%5Btextonly%5D=Yes&MHG%5Bspl%5D=yes&MHG%5Btournament%5D=2023%20Season%20World%20Championship/Main%20Event&_run=)

Code is provided under the `lol` directory so as to keep this notebook slimmer.

## The Data

We will load all of the tournament data and concatenate them into a single, monolithic dataframe. We will also load the hero data used to augment the match data.

In [2]:
import lol 
champs = lol.get_champ_data()
top_lane = lol.get_top_data()

ModuleNotFoundError: No module named 'lol'

In [ ]:
champs.info()

In [ ]:
champs.sample()

In [ ]:
top_lane.info()

In [ ]:
top_lane.sample()

I will first start with the `champs` data. The Data Frame contains a significant amount of data; however, I am going to focus on the `picks` column and `bans` column as well as the `winrate` column. 

## The Draft Phase (Picks & Bans)

I will start by looking at the presense of the champions. In a professional match of League of Legends, the teams take turns banning and choosing champions. This part of the match is called the *draft phase*. When a champion is banned, neither team is allowed to select this champion. This is the same for when a team selects a champion too (there are no duplicate champions allowed). I am looking to see which champions are deemed to be more vaulable compared to others. Thus, I will be looking at the Presence column. This column shows the percentage that the champion was present in all of the drafts.

In the `Picks` and `Bans` columns, it shows the number of times a champion was picked or banned during the draft phase. We can also observe this data to see how much priority a champion had during the World Championships and how this changed throughout the years.

To begin I will filter the data and sort it by only looking at the `Presense`, `Picks`, `Bans`, and `Year` columns.

In [ ]:
import pandas as pd

# pulling the columns that I need for my plot
presence_data = champs.loc[:,('Champion', 'Picks', 'Bans', 'Presence', 'Year')]


Now that I have the data I need, I am creating a slider so anyone can sort the data as they please.

In [ ]:
import panel as pn 

# changing 'Presence' column to an integer type and removing the '%' symbol
presence_data['Presence'] = presence_data['Presence'].str.replace('%', ' ').astype(int)

# creating the tabulator for the presence_data
presence_data_tb = pn.widgets.Tabulator(
    presence_data,
    page_size=15,
    pagination='local',
    selectable=False,
    disabled=False
)
# provide a slider to filter out heroes with low presence counts
pick_slider = pn.widgets.IntSlider(
    name='Minimum Presence',
    start=0,
    end=int(presence_data.Presence.max()),
    step=1
)

@pn.depends(pick_slider, watch=True)
def filtered_champ_picks(value):
    presence_data_tb._filters = []
    presence_data_tb.add_filter((value, pick_slider.end), 'Presence')

pn.Column(pick_slider, presence_data_tb)


If you sort the Dataframe by highest to lowest Presence you can see that most of the champions who have a high presence have more bans than picks. The reason for their high presence in the World Championship is due to teams being afraid to play against these champions so they ban it instead of trying to counter pick it. 

Looking at the 5 champions with 100% presence, we can see that two of them, Kalista and Pantheon, have a 100% ban rate. This means that these champions were banned every single game at the World Championship. They were never picked, and never not banned. 

In [ ]:
import pandas as pd
import holoviews as hv
import hvplot.pandas
import panel as pn

def scatter_plot(champion):
    selected_data = presence_data[presence_data['Champion'] == champion].copy()

# scale the size of Presence by an addition 1000 due to the small sizing of the data points with regular Presense size.
    selected_data['Scaled_Presence'] = selected_data['Presence'] + 1000

    plot = selected_data.hvplot.scatter(
        x='Picks',
        y='Bans',
        s='Scaled_Presence',
        c='Year',
        cmap='viridis',
        clabel='By Year',
        hover_cols=['Champion']
    ).opts(
        xlim=(-5,60),
        ylim=(-5,80),
        clim=(2015, 2023),
        title=f'Presence of {champion} in World Championship',
        xlabel='Picks',
        ylabel='Bans',
        height=450,
        width=800,
    )
    return plot

champion_names = sorted(presence_data['Champion'].unique())

champion_selector = pn.widgets.Select(options=champion_names, name='Select Champion')

@pn.depends(champion=champion_selector.param.value)
def update_plot(champion):
    return scatter_plot(champion)

graph = pn.Row(
    pn.Column(champion_selector),
    pn.Column(update_plot)
)

graph


When using the above Champion Selector, you can see the `Picks` and `Bans` for each champion. They are color coded by the `Year` of the World Championship. For some champions, there are multiple circles overlapping due to them having the same 0 `Picks` and `Bans`, thus meaning 0 `Presence`. We do notice that the further a champ is to the top or right of the graph, the higher the Presence is for that champion. When hovering over the points, you can see the specific data of that champion. I had to scale the Presence by an additional 1000 (`Scaled_Presence`) due to the data points being too small when it's scaled by the Presence Column.

## Best Top Laner of World Championship 2023 

In League of Legends, there are three lanes: Top, Middle, and Bottom. In this years World Championship, there was a lot of press and talk about the Top Lane players. Specifically, a Korean Top Laner by the name of Zeus. Everyone was discussing how he was dominating every player he came across throughout the tournament. His team ended up winning the World Championship and he was given the Player MVP Trophy.

So, I will be looking at all of the Top Laners in the 2023 World Championship Tournament and comparing their statistics to Zeus' to see if he was dominating like the press and media said. 

In [ ]:
# removing the percentage symbol from winrate and changing it to a numeric string type
top_lane['Win rate'] = pd.to_numeric(top_lane['Win rate'].str.replace('%', ''))

# creating the tabulator for the top_lane data
top_lane_tb = pn.widgets.Tabulator(
    top_lane,
    pagination='local',
    selectable=False,
    disabled=False
)
# provide a slider to filter out low KDA
top_slider = pn.widgets.FloatSlider(
    name='Kill Death Assist',
    start=0,
    end=float(top_lane.KDA.max()),
    step=0.1
)

@pn.depends(top_slider, watch=True)
def filtered_top_picks(value):
    top_lane_tb._filters = []
    top_lane_tb.add_filter((value, top_slider.end), 'KDA')

pn.Column(top_slider, top_lane_tb)


The above data frame has a slider where you can sort by Kill Death Assist (KDA). KDA means the ratio of the sum of kills and assists, over deaths. Obviously, a player would want higher kills/assists than deaths in any game. We can see that most of the players have a KDA in the range 2-3 give or take. However, when looking at Zeus' KDA at a score of 5.0, this is an example of how dominate he really was. This means that for every 5 kills he got, he only died 1 time. Comparing this to the other players, who have around a 2-3 KDA, this means thats they only got 2-3 kills per 1 death. Zeus' KDA is doubled compared to some of the other players.

I will now make a plot comparing the Gold per Minute (GPM) and Creep Score per Minute (CSM). 

In [ ]:
# making scatter plot of top laners
top_data = top_lane.hvplot.scatter(
        x='GPM',
        y='CSM',
        s=1000,
        c='Win rate',
        cmap='plasma',
        clabel='Win Rate',
        hover_cols=['Player']
    ).opts(
        xlim=(350,460),
        ylim=(7,9.5),
        clim=(0.00, 100.00),
        title=f'GPM and CSM effect on Win Rate',
        xlabel='Gold Per Minute',
        ylabel='Creep Score Per Minute',
        height=480,
        width=900,
    )

top_data

From this graph, we can clearly see that Zeus did indeed dominate his competitors. All of the other players are all grouped near each other whereas Zeus is all the way in the top right of the graph. This is mainly due to his amount of Gold per Minute which is an additional 49 gold above the next closest player. Also, looking at the win rate of the players, they all are a purple/pink hue whereas Zeus is a yellow color. This is due to his win rate being significantly higher compared to the other players. 

Even when looking at other statistics in the data frame above, we can see that Zeus is in the top 3 of all the good statistics such as DPM (Damage per Minute), GD@15 (Average Gold Differential at 15 minutes), CSD@15 (Average CS Score at 15 minutes), and Solo Kills. All of these statistics are focused on the individual and not the team. The teammates have a minor effect on this data compared to the player, who is the primary factor influencing it.

## Conclusion

In this notebook, we have looked at the statistics of champion data through the League of Legends World Championships from 2015-2023. Despite this being the most important tournament in the world, there are many other smaller and regional tournaments that were not looked at. The champion data over the past 9 years has provided us with some insight on which characters are more important in the World Championship meta, as well as which champions are neglected and never played. 

We also took a look at the Top Lane Players in the 2023 World Championship tournament, where we saw that 1 player (Zeus) stood out compared to the rest. Because of his amazing performances and large player gap, it is clear to anyone that he deserved to win the 2023 MVP Trophy. This process can be similarly done to all lanes and players throughout all of the World Championships.